In [1]:
import numpy as np
import tensorflow as tf

#DATA : https://github.com/suriyadeepan/practical_seq2seq/tree/master/datasets
from datasets.twitter import data
import data_utils

#### Import preprocessed data

In [2]:
metadata, id_q, id_a = data.load_data(PATH='datasets/twitter/')
(trainX, trainY), (testX, testY), (validX, validY) = data_utils.split_dataset(id_q, id_a)

In [3]:
trainX[:5]

array([[   7,  106,    3,  616,  214, 1188,  164, 1400, 4744,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0],
       [ 100,    4,  467, 1312,  285,   36,    2, 2187,   10,  842,    2,
           1, 1014,    1,  976, 4362,  467, 3840,   16, 1585],
       [   1,    1, 1741,    5,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0],
       [ 745,  282,  745,    8,  101,   85,  347,  320,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0],
       [1185,    4,   40,  327,    7,  162,  888,   53,    4,  437,   15,
          44,    7,  125,  207,  220,    5,    0,    0,    0]])

## Parameters

In [4]:
xseq_len = trainX.shape[-1]
yseq_len = trainY.shape[-1]
batch_size = 16
xvocab_size = len(metadata['idx2w'])
yvocab_size = xvocab_size
emb_dim = 1024

In [14]:
import seq2seq_wrapper

In [15]:
import importlib
importlib.reload(seq2seq_wrapper)

<module 'seq2seq_wrapper' from 'C:\\Users\\N1801369H\\Documents\\Seq2Seq\\seq2seq_wrapper.py'>

In [16]:
tf.reset_default_graph()
model = seq2seq_wrapper.Seq2Seq(xseq_len = xseq_len,
                               yseq_len = yseq_len,
                               xvocab_size = xvocab_size,
                               yvocab_size = yvocab_size,
                               ckpt_path = 'ckpt/twitter/',
                               emb_dim = emb_dim,
                               num_layers = 3)

<log> Building Graph </log>

In [17]:
val_batch_gen = data_utils.rand_batch_gen(validX, validY, 256)
test_batch_gen = data_utils.rand_batch_gen(testX, testY, 256)
train_batch_gen = data_utils.rand_batch_gen(trainX, trainY, batch_size)

### Training

In [18]:
sess = model.train(train_batch_gen, val_batch_gen)


<log> Training started </log>

Model saved to disk at iteration #100
val   loss : 3.403190

Model saved to disk at iteration #200
val   loss : 3.361432

Model saved to disk at iteration #300
val   loss : 3.298964

Model saved to disk at iteration #400
val   loss : 3.242165

Model saved to disk at iteration #500
val   loss : 3.093632
Interrupted by user at iteration 580


### Restore checkpoint

In [19]:
sess = model.restore_last_session()

INFO:tensorflow:Restoring parameters from ckpt/twitter/seq2seq_model.ckpt-500


In [20]:
input_ = test_batch_gen.__next__()[0]
output = model.predict(sess, input_)
print(output.shape)

(256, 20)


In [21]:
replies = []
for ii,oi in zip(input_.T, output):
    q = data_utils.decode(sequence=ii, lookup=metadata['idx2w'], separator=' ')
    decoded = data_utils.decode(sequence=oi, lookup=metadata['idx2w'], separator=' ').split()
    if decoded.count('unk') == 0:
        if decoded not in replies:
            print('q : [{0}]; a:[{1}]'.format(q, ' '.join(decoded)))
            replies.append(decoded)

q : [never mind i found it]; a:[i i i]
q : [how much are the touch unk going to be looks like i just spent that money somewhere else instead]; a:[]
q : [more hillary right now i was thinking of third party then i remembered that unk halloween episode about aliens]; a:[i i i i]
q : [200 more white people have been shot by police this year than blacks stop trying to start a race war]; a:[i]
